<a href="https://colab.research.google.com/github/abhijith2021/Fetal-health-prediction-from-cardiotocographic-data/blob/main/ANN_FETUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from kerastuner import HyperModel, RandomSearch

ctg_data = pd.read_csv("/content/merged_signals.csv")



ctg_features = ["seconds", "FHR", "UC"]

label_encoder = LabelEncoder()
ctg_data["FHR"] = label_encoder.fit_transform(ctg_data["FHR"])

X_train, X_val, y_train, y_val = train_test_split(
    ctg_data[ctg_features],
    ctg_data["FHR"],
    test_size=0.2,
    random_state=42,
)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=128, step=32),
                    activation='relu', input_shape=(X_train_scaled.shape[1],)))
    model.add(Dense(units=hp.Int('second_layer_units', min_value=16, max_value=64, step=16),
                    activation='relu'))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))

    model.compile(loss="sparse_categorical_crossentropy", optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
                  metrics=['accuracy'])
    return model

tuner = RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1,
    executions_per_trial=1,
    directory="/content/hyperparameter_tuning_results",
    project_name="fetal_heartbeat_predictor_classification",
    **{"max_consecutive_failed_trials": 5}
)

tuner.search(X_train_scaled, y_train, epochs=10, validation_data=(X_val_scaled, y_val), verbose=2)


model = Sequential()
model.add(Dense(128, activation="relu", input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(64, activation="relu"))
model.add(Dense(len(label_encoder.classes_), activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_val_scaled, y_val))

y_pred_prob = model.predict(X_val_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

model.save("/content/fetal_heartbeat_predictor_classification.h5")


Trial 1 Complete [00h 16m 23s]
val_accuracy: 0.5126862525939941

Best val_accuracy So Far: 0.5126862525939941
Total elapsed time: 00h 16m 23s
Epoch 1/20
26215/26215 [==============================] - 115s 4ms/step - loss: 2.6923 - accuracy: 0.3149 - val_loss: 2.0878 - val_accuracy: 0.3954
Epoch 2/20
26215/26215 [==============================] - 119s 5ms/step - loss: 1.9143 - accuracy: 0.4202 - val_loss: 1.7780 - val_accuracy: 0.4502
Epoch 3/20
26215/26215 [==============================] - 115s 4ms/step - loss: 1.6779 - accuracy: 0.4694 - val_loss: 1.5907 - val_accuracy: 0.4991
Epoch 4/20
26215/26215 [==============================] - 118s 5ms/step - loss: 1.5347 - accuracy: 0.5028 - val_loss: 1.4609 - val_accuracy: 0.5324
Epoch 5/20
26215/26215 [==============================] - 115s 4ms/step - loss: 1.4414 - accuracy: 0.5250 - val_loss: 1.3776 - val_accuracy: 0.5398
Epoch 6/20
26215/26215 [==============================] - 114s 4ms/step - loss: 1.3732 - accuracy: 0.5426 - val_loss: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7086
Precision: 0.6579
Recall: 0.7086
F1 Score: 0.6462


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install keras-tuner
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from kerastuner import HyperModel, RandomSearch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

ctg_data = pd.read_csv("/content/merged_signals.csv")

ctg_features = ["seconds", "FHR", "UC"]
target = "FHR"

label_encoder = LabelEncoder()
ctg_data[target] = label_encoder.fit_transform(ctg_data[target])

X_train, X_val, y_train, y_val = train_test_split(
    ctg_data[ctg_features],
    ctg_data[target],
    test_size=0.2,
    random_state=42,
)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

X_train_scaled = np.expand_dims(X_train_scaled, axis=2)
X_val_scaled = np.expand_dims(X_val_scaled, axis=2)


def build_model(hp):
    model = Sequential()
    model.add(GRU(units=hp.Int('units', min_value=32, max_value=128, step=32),
                  return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)))
    model.add(GRU(units=hp.Int('second_layer_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']), metrics=['accuracy'])
    return model

tuner = RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1,
    executions_per_trial=1,
    directory="/content/hyperparameter_tuning_results",
    project_name="fetal_heartbeat_predictor",
    **{"max_consecutive_failed_trials": 5}
)

tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_val_scaled, y_val), verbose=2)

model = Sequential()
model.add(GRU(64, return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)))
model.add(GRU(32))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_val_scaled, y_val))

y_pred_prob = model.predict(X_val_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='weighted')
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Trial 1 Complete [01h 24m 26s]
val_accuracy: 0.7055575251579285

Best val_accuracy So Far: 0.7055575251579285
Total elapsed time: 01h 24m 26s
Epoch 1/10
26215/26215 [==============================] - 209s 8ms/step - loss: 2.8145 - accuracy: 0.3191 - val_loss: 1.9913 - val_accuracy: 0.4387
Epoch 2/10
26215/26215 [==============================] - 210s 8ms/step - loss: 1.8465 - accuracy: 0.4602 - val_loss: 1.7117 - val_accuracy: 0.5272
Epoch 3/10
26215/26215 [==============================] - 210s 8ms/step - loss: 1.6624 - accuracy: 0.5122 - val_loss: 1.5798 - val_accuracy: 0.5674
Epoch 4/10
26215/26215 [==============================] - 215s 8ms/step - loss: 1.5451 - accuracy: 0.5494 - val_loss: 1.4790 - val_accuracy: 0.5824
Epoch 5/10
26215/26215 [==============================] - 221s 8ms/step - loss: 1.4456 - accuracy: 0.5768 - val_loss: 1.3962 - val_accuracy: 0.6040
Epoch 6/10
26215/26215 [==============================] - 215s 8ms/step - loss: 1.3762 - accuracy: 0.5964 - val_loss: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
